<a href="https://colab.research.google.com/github/fahizakaruppanthodi/Decoding-IMDB-Movie-Reviews-/blob/main/Decoding_IMDB_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

In [ ]:
x =  pd.read_csv("/content/train.csv", encoding='latin1', on_bad_lines='skip', engine='python')
y =  pd.read_csv("/content/test.csv", encoding='latin1', on_bad_lines='skip', engine='python')

In [ ]:
x

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg
...,...,...
22024,"At first the movie seemed to be doing great, t...",neg
22025,"The Legend of Zu, as I saw it, was a very inte...",pos
22026,Taking old collection of stories poses a chall...,pos
22027,Strangely erotic schlock Gothic horror that wi...,pos


In [ ]:
y

,text,sentiment
0,"My daughter liked it but I was aghast, that a ...",neg
1,I... No words. No words can describe this. I w...,neg
2,this film is basically a poor take on the old ...,neg
3,"This is a terrible movie, and I'm not even sur...",neg
4,First of all this movie is a piece of reality ...,pos
...,...,...
20134,****SOME SPOILERS****<br /><br />There's nothi...,neg
20135,I bought this 'film' from a gas station in the...,neg
20136,I remember watching this movie over and over a...,pos
20137,I think the cards were stacked against Webmast...,neg


In [ ]:
# Combines the training and test DataFrames
df=pd.concat([x,y],ignore_index=True)
df

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg
...,...,...
42163,****SOME SPOILERS****<br /><br />There's nothi...,neg
42164,I bought this 'film' from a gas station in the...,neg
42165,I remember watching this movie over and over a...,pos
42166,I think the cards were stacked against Webmast...,neg


In [ ]:
df.head()

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg


In [ ]:
df.tail()

,text,sentiment
42163,****SOME SPOILERS****<br /><br />There's nothi...,neg
42164,I bought this 'film' from a gas station in the...,neg
42165,I remember watching this movie over and over a...,pos
42166,I think the cards were stacked against Webmast...,neg
42167,`Skenbart' is one of the funniest movies to no...,pos


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
neg,21098
pos,21070


In [ ]:
import nltk
import re

In [ ]:
nltk.download("wordnet")      # For lemmatization (WordNetLemmatizer)
nltk.download("stopwords")    # Common English stopwords (like "the", "is", etc.)
nltk.download("punkt")        # Tokenizer models (splits text into words/sentences)
nltk.download("omw-1.4")      # WordNet's multilingual support (used with wordnet)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Also valid, but less safe if column names have spaces or special characters
sms=df.text
sms

,text
0,"Now, I won't deny that when I purchased this o..."
1,"The saddest thing about this ""tribute"" is that..."
2,Last night I decided to watch the prequel or s...
3,I have to admit that i liked the first half of...
4,I was not impressed about this film especially...
...,...
42163,****SOME SPOILERS****<br /><br />There's nothi...
42164,I bought this 'film' from a gas station in the...
42165,I remember watching this movie over and over a...
42166,I think the cards were stacked against Webmast...


In [ ]:
from nltk.tokenize import TweetTokenizer   #Create the tokenizer
tk=TweetTokenizer()                       # Join tokens back into a string

In [ ]:
# Replaces any character that is not a letter or digit with a space.
sms=sms.apply(lambda x:re.sub('[^a-zA-Z0-9]+'," ",x))
sms

,text
0,Now I won t deny that when I purchased this of...
1,The saddest thing about this tribute is that a...
2,Last night I decided to watch the prequel or s...
3,I have to admit that i liked the first half of...
4,I was not impressed about this film especially...
...,...
42163,SOME SPOILERS br br There s nothing in this m...
42164,I bought this film from a gas station in the 3...
42165,I remember watching this movie over and over a...
42166,I think the cards were stacked against Webmast...


In [ ]:
# SnowballStemmer is used in Natural Language Processing (NLP) to reduce words to their root form, known as stemming.
from nltk.stem import SnowballStemmer #Initialize the stemmer and tokenizer
stemmer=SnowballStemmer("english")
sms=sms.apply(lambda x:[stemmer.stem(i.lower()) for i in tk.tokenize(x)]).apply(lambda x:" ".join(x)) # Apply stemming to each token in the SMS messages
sms

,text
0,now i won t deni that when i purchas this off ...
1,the saddest thing about this tribut is that al...
2,last night i decid to watch the prequel or sha...
3,i have to admit that i like the first half of ...
4,i was not impress about this film especi for t...
...,...
42163,some spoiler br br there s noth in this movi t...
42164,i bought this film from a gas station in the 3...
42165,i rememb watch this movi over and over again w...
42166,i think the card were stack against webmast be...


In [ ]:
# (the", "is", "in", "at", "a", "I", "you", "he", "and", "to") all stop words are remove
from nltk.corpus import stopwords
stop_words=stopwords.words("english") #Filters out English stopwords
sms=sms.apply(lambda x:[i for i in tk.tokenize(x) if i not in stop_words]).apply(lambda x:" ".join(x))
sms

,text
0,deni purchas ebay high expect incred print wor...
1,saddest thing tribut almost singer includ othe...
2,last night decid watch prequel shall say call ...
3,admit like first half sleeper look good act ev...
4,impress film especi fact went cinema famili go...
...,...
42163,spoiler br br noth movi alreadi seen brows net...
42164,bought film gas station 3 99 bin one reason bi...
42165,rememb watch movi kid love whilst watch recent...
42166,think card stack webmast becaus right start it...


In [ ]:
# Vectorization means converting text into numbers,
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer() #nitialize TF-IDF vectorizer
train_data=vec.fit_transform(sms) #Fit and transform the SMS text data
train_data                        #Check the shape of the result

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4038166 stored elements and shape (42168, 64721)>

In [ ]:
print(train_data)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4038166 stored elements and shape (42168, 64721)>
  Coords	Values
  (0, 15275)	0.14379602766510097
  (0, 45585)	0.14121297988778853
  (0, 17728)	0.3583849497795395
  (0, 26289)	0.07739833835288351
  (0, 19445)	0.08001945010511193
  (0, 28127)	0.10150704038955996
  (0, 44976)	0.13836120453415124
  (0, 63408)	0.06250004855796575
  (0, 35683)	0.11307768662525446
  (0, 12318)	0.07945628633974415
  (0, 18596)	0.1437109783738696
  (0, 27138)	0.07648112264083254
  (0, 53102)	0.10177411422408321
  (0, 16080)	0.08928751182210866
  (0, 3675)	0.15872707216440365
  (0, 21444)	0.0823052504183618
  (0, 12447)	0.24308961090338616
  (0, 2418)	0.41170457094238583
  (0, 61116)	0.050072780101937824
  (0, 15853)	0.11319066912648604
  (0, 62157)	0.04855427202738049
  (0, 23820)	0.15215351255417886
  (0, 52572)	0.12330407927024788
  (0, 55379)	0.07857293780621036
  (0, 34881)	0.10121524103123379
  :	:
  (42167, 28595)	0.1502966031720783
  (42167,

In [ ]:
train_data.shape

(42168, 64721)

In [ ]:
y=df['sentiment'].values
y

array(['neg', 'neg', 'neg', ..., 'pos', 'neg', 'pos'], dtype=object)

In [ ]:
# 70% train 30% test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_data,y,test_size=0.2,random_state=42)
x_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3233326 stored elements and shape (33734, 64721)>

In [ ]:
x=train_data

In [ ]:
# Create the SVM model
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
y_pred

array(['neg', 'neg', 'neg', ..., 'neg', 'neg', 'neg'], dtype=object)